## RAW DATA TO MONGODB STAGING

In [1]:
import json
import pprint
import numpy as np
import pandas as pd
from datetime import datetime

import pymongo
from pymongo import MongoClient

# import findspark
# findspark.init('C:/BigDataLocalSetup/spark')
import pyspark
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook2").\
        config("spark.executor.memory", "1g").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        getOrCreate()

In [2]:
client = MongoClient("mongodb://127.0.0.1:27017")
db = client.MED_EX_CHAIN

gpo = db.GPO
product = db.PRODUCT
hospital = db.HOSPITAL
user_roles = db.USER_ROLES
distributor = db.DISTRIBUTOR
manufacturer = db.MANUFACTURER
correlation = db.CORRELATION_DATA
product_similarity = db.PRODUCT_SIMILARITY

In [3]:
raw_data_path = "../Data/SampleData/Schema - Master Data/"

### Distributor Raw Staging

In [4]:
distributor.insert_one({'DISTRIBUTOR_ID': 0, 'VERSION': 0})

In [5]:
distributor_csv = spark.read.format("mongo").option("uri", 
                                "mongodb://127.0.0.1:27017/medexchain-catalog.distributor").load()
distributor_csv.createOrReplaceTempView('distributor_raw_view')

In [6]:
distributor_csv = distributor_csv.select(col('_id'),
                                                   col('org_type'),
                                                   col('name'),
                                                   col('short_name'),
                                                   col('source_identifier'),
                                                   col('country_of_origin'),
                                                   col('address'))

AnalysisException: cannot resolve '`_id`' given input columns: [];
'Project ['_id, 'org_type, 'name, 'short_name, 'source_identifier, 'country_of_origin, 'address]
+- Relation[] MongoRelation(MongoRDD[0] at RDD at MongoRDD.scala:51,Some(StructType()))


In [7]:
distributor_csv = distributor_csv.withColumnRenamed('_id', 'DISTRIBUTOR_ID').\
                                            withColumnRenamed('org_type', 'DISTRIBUTOR_ORG_TYPE').\
                                            withColumnRenamed('name', 'DISTRIBUTOR_NAME').\
                                            withColumnRenamed('short_name', 'DISTRIBUTOR_SHORT_NAME').\
                                            withColumnRenamed('source_identifier', 'SOURCE_IDENTIFIER').\
                                            withColumnRenamed('country_of_origin', 'COUNTRY_OF_ORIGIN').\
                                            withColumnRenamed('address', 'DISTRIBUTOR_ADDRESS')
#                                             withColumnRenamed('UM', 'UOM').\
#                                             withColumnRenamed('base_price', 'BASE_PRICE').\
#                                             withColumnRenamed(' PRODUCT_TYPE_ID ', 'PRODUCT_TYPE_ID').\
#                                             withColumnRenamed('name', 'NAME').\
#                                             withColumnRenamed('short_description', 'SHORT_DESCRIPTION').\
#                                             withColumnRenamed('status', 'PRODUCT_STATUS').\
#                                             withColumnRenamed('is_pack', 'IS_PACK').\
#                                             withColumnRenamed('units_in_pack', 'UNITS_IN_PACK').\
#                                             withColumnRenamed('category', 'PRODUCT_CATEGORY').\
#                                             withColumnRenamed('mfg_date', 'MFG_DATE').\
#                                             withColumnRenamed('expiry_date', 'EXPIRY_DATE').\
#                                             withColumnRenamed('image', 'PRODUCT_IMAGE')

In [8]:
distributor_data = spark.read.format("mongo").option("uri", 
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.DISTRIBUTOR").load()
distributor_data.createOrReplaceTempView('distributor_view')

In [9]:
for i in distributor_csv.toJSON().collect():
    new_record = json.loads(i)
    distributor_id = str(new_record['DISTRIBUTOR_ID'])
    
    query = "select * from distributor_view \
             where DISTRIBUTOR_ID = '" + distributor_id + "' and\
                   VERSION = (select max(VERSION) from distributor_view \
             where DISTRIBUTOR_ID = '" + distributor_id + "')"

    available_data = spark.sql(query)
    print(distributor_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        distributor.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['DISTRIBUTOR_NAME'] != available_data.rdd.collect()[0]['DISTRIBUTOR_NAME'])|\
         (new_record['DISTRIBUTOR_ADDRESS'] != available_data.rdd.collect()[0]['DISTRIBUTOR_ADDRESS'])|\
         (new_record['DISTRIBUTOR_SHORT_NAME'] != available_data.rdd.collect()[0]['DISTRIBUTOR_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.
         
         distributor.update_many({'DISTRIBUTOR_ID': distributor_id, 
                                  'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                           'END_DATE': str(datetime.now())}})
        

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         distributor.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.             
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        distributor.update_one({'DISTRIBUTOR_ID': distributor_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                            {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


In [10]:
distributor_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.DISTRIBUTOR").load()
distributor_data.show()                            

+-----------------+-------------------+--------------+----------------+--------------------+----------------------+----------+--------------------+----------------+-----------------+--------------------+-------+--------------------+
|COUNTRY_OF_ORIGIN|DISTRIBUTOR_ADDRESS|DISTRIBUTOR_ID|DISTRIBUTOR_NAME|DISTRIBUTOR_ORG_TYPE|DISTRIBUTOR_SHORT_NAME|  END_DATE|      LAST_UPDATE_DT|LAST_UPDATE_USER|SOURCE_IDENTIFIER|          START_DATE|VERSION|                 _id|
+-----------------+-------------------+--------------+----------------+--------------------+----------------------+----------+--------------------+----------------+-----------------+--------------------+-------+--------------------+
|             null|               null|             0|            null|                null|                  null|      null|                null|            null|             null|                null|    0.0|{61082d5121e1ff33...|
|               US|        D ADDR 1001|         D1001|        mckess

### GPO Raw staging

In [11]:
gpo.insert_one({'GPO_ID':0, 'VERSION': 0})

In [12]:
gpo_csv = spark.read.format("mongo").option("uri", 
                                "mongodb://127.0.0.1:27017/medexchain-catalog.gpo").load()
gpo_csv.createOrReplaceTempView('gpo_raw_view')

In [13]:
gpo_csv = gpo_csv.select(col('_id'),
                         col('org_type'),
                         col('name'),
                         col('short_name'),
                         col('source_identifier'),
                         col('country_of_origin'),
                         col('address'))

AnalysisException: cannot resolve '`_id`' given input columns: [];
'Project ['_id, 'org_type, 'name, 'short_name, 'source_identifier, 'country_of_origin, 'address]
+- Relation[] MongoRelation(MongoRDD[28] at RDD at MongoRDD.scala:51,Some(StructType()))


In [ ]:
gpo_csv = gpo_csv.withColumnRenamed('_id', 'GPO_ID').\
                  withColumnRenamed('org_type', 'GPO_ORG_TYPE').\
                  withColumnRenamed('name', 'GPO_NAME').\
                  withColumnRenamed('short_name', 'GPO_SHORT_NAME').\
                  withColumnRenamed('source_identifier', 'SOURCE_IDENTIFIER').\
                  withColumnRenamed('country_of_origin', 'COUNTRY_OF_ORIGIN').\
                  withColumnRenamed('address', 'GPO_ADDRESS')
#                                             withColumnRenamed('UM', 'UOM').\
#                                             withColumnRenamed('base_price', 'BASE_PRICE').\
#                                             withColumnRenamed(' PRODUCT_TYPE_ID ', 'PRODUCT_TYPE_ID').\
#                                             withColumnRenamed('name', 'NAME').\
#                                             withColumnRenamed('short_description', 'SHORT_DESCRIPTION').\
#                                             withColumnRenamed('status', 'PRODUCT_STATUS').\
#                                             withColumnRenamed('is_pack', 'IS_PACK').\
#                                             withColumnRenamed('units_in_pack', 'UNITS_IN_PACK').\
#                                             withColumnRenamed('category', 'PRODUCT_CATEGORY').\
#                                             withColumnRenamed('mfg_date', 'MFG_DATE').\
#                                             withColumnRenamed('expiry_date', 'EXPIRY_DATE').\
#                                             withColumnRenamed('image', 'PRODUCT_IMAGE')

In [14]:
gpo_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.GPO").load()
gpo_data.createOrReplaceTempView('gpo_view')

In [15]:
for i in gpo_csv.toJSON().collect():
    new_record = json.loads(i)
    gpo_id = str(new_record['GPO_ID'])

    query = "select * from gpo_view \
             where GPO_ID = '" + gpo_id + "' and\
                   VERSION = (select max(VERSION) from gpo_view \
             where GPO_ID = '" + gpo_id + "')"
    
    available_data = spark.sql(query)
    print(gpo_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        gpo.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['GPO_NAME'] != available_data.rdd.collect()[0]['GPO_NAME'])|\
         (new_record['GPO_ADDRESS'] != available_data.rdd.collect()[0]['GPO_ADDRESS'])|\
         (new_record['GPO_SHORT_NAME'] != available_data.rdd.collect()[0]['GPO_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.

         gpo.update_many({'GPO_ID': gpo_id, 
                          'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                         {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                   'END_DATE': str(datetime.now())}})

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         gpo.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                 
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        gpo.update_one({'GPO_ID': gpo_id, 
                             'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                             {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


In [16]:
gpo_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.GPO").load()
gpo_data.show()                            

+-----------------+----------+-----------+------+----------+------------+--------------+--------------------+----------------+-----------------+--------------------+-------+--------------------+
|COUNTRY_OF_ORIGIN|  END_DATE|GPO_ADDRESS|GPO_ID|  GPO_NAME|GPO_ORG_TYPE|GPO_SHORT_NAME|      LAST_UPDATE_DT|LAST_UPDATE_USER|SOURCE_IDENTIFIER|          START_DATE|VERSION|                 _id|
+-----------------+----------+-----------+------+----------+------------+--------------+--------------------+----------------+-----------------+--------------------+-------+--------------------+
|             null|      null|       null|     0|      null|        null|          null|                null|            null|             null|                null|    0.0|{61082d8721e1ff33...|
|               US|9999-12-31|G addr 1004| G1004|GPO_NAME13|    Hospital| gpo last name|2021-08-02 23:08:...|            USER|               S4|2021-08-02 23:08:...|    1.0|{61082d8d21e1ff33...|
|               US|9999-1

### Hospital Raw Staging

In [17]:
hospital.insert_one({'HOSPITAL_ID':0, 'VERSION': 0})

In [18]:
# file_name = 'Hospital.csv'
# file_name = input('Enter File Name: ')
# hospital_csv = spark.read.csv(raw_data_path + file_name, header = True).dropna(how = "all")
# hospital_csv.show()

In [19]:
hospital_csv = spark.read.format("mongo").option("uri", 
                                "mongodb://127.0.0.1:27017/medexchain-catalog.hospital").load()
hospital_csv.createOrReplaceTempView('hospital_raw_view')

In [20]:
hospital_csv = hospital_csv.select(col('_id'),
#                          col('org_type'),
                         col('name'),
                         col('short_name'),
                         col('source_identifier'),
                         col('country_of_origin'),
                         col('address'))

AnalysisException: cannot resolve '`_id`' given input columns: [];
'Project ['_id, 'name, 'short_name, 'source_identifier, 'country_of_origin, 'address]
+- Relation[] MongoRelation(MongoRDD[56] at RDD at MongoRDD.scala:51,Some(StructType()))


In [21]:
# hospital_csv = hospital_csv.withColumnRenamed('_id', 'HOSPITAL_ID').\
#                   withColumnRenamed('org_type', 'HOSPITAL_TYPE')
#                   withColumnRenamed('name', 'HOSPITAL_NAME').\
#                   withColumnRenamed('short_name', 'HOSPITAL_SHORT_NAME').\
#                   withColumnRenamed('source_identifier', 'SOURCE_IDENTIFIER').\
#                   withColumnRenamed('country_of_origin', 'COUNTRY_OF_ORIGIN').\
#                   withColumnRenamed('address', 'HOSPITAL_ADDRESS')
#                                             withColumnRenamed('UM', 'UOM').\
#                                             withColumnRenamed('base_price', 'BASE_PRICE').\
#                                             withColumnRenamed(' PRODUCT_TYPE_ID ', 'PRODUCT_TYPE_ID').\
#                                             withColumnRenamed('name', 'NAME').\
#                                             withColumnRenamed('short_description', 'SHORT_DESCRIPTION').\
#                                             withColumnRenamed('status', 'PRODUCT_STATUS').\
#                                             withColumnRenamed('is_pack', 'IS_PACK').\
#                                             withColumnRenamed('units_in_pack', 'UNITS_IN_PACK').\
#                                             withColumnRenamed('category', 'PRODUCT_CATEGORY').\
#                                             withColumnRenamed('mfg_date', 'MFG_DATE').\
#                                             withColumnRenamed('expiry_date', 'EXPIRY_DATE').\
#                                             withColumnRenamed('image', 'PRODUCT_IMAGE')

In [22]:
hospital_csv = hospital_csv.withColumnRenamed('_id', 'HOSPITAL_ID').\
                  withColumnRenamed('name', 'HOSPITAL_NAME').\
                  withColumnRenamed('short_name', 'HOSPITAL_SHORT_NAME').\
                  withColumnRenamed('source_identifier', 'SOURCE_IDENTIFIER').\
                  withColumnRenamed('country_of_origin', 'COUNTRY_OF_ORIGIN').\
                  withColumnRenamed('address', 'HOSPITAL_ADDRESS')

In [23]:
hospital_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.HOSPITAL").load()
hospital_data.createOrReplaceTempView('hospital_view')

In [24]:
for i in hospital_csv.toJSON().collect():
    new_record = json.loads(i)
    hospital_id = str(new_record['HOSPITAL_ID'])

    query = "select * from hospital_view \
             where HOSPITAL_ID = '" + hospital_id + "' and\
                   VERSION = (select max(VERSION) from hospital_view \
             where HOSPITAL_ID = '" + hospital_id + "')"
    
    available_data = spark.sql(query)
    print(hospital_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        hospital.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['HOSPITAL_NAME'] != available_data.rdd.collect()[0]['HOSPITAL_NAME'])|\
         (new_record['HOSPITAL_ADDRESS'] != available_data.rdd.collect()[0]['HOSPITAL_ADDRESS'])|\
         (new_record['HOSPITAL_SHORT_NAME'] != available_data.rdd.collect()[0]['HOSPITAL_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.
         
         hospital.update_many({'HOSPITAL_ID': hospital_id, 
                               'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                              {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                        'END_DATE': str(datetime.now())}})

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         hospital.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        hospital.update_one({'HOSPITAL_ID': hospital_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


In [25]:
hospital_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.HOSPITAL").load()
hospital_data.show()                             


+-----------------+----------+----------------+-----------+--------------------+-------------------+-------------------+--------------------+----------------+-----------------+--------------------+-------+--------------------+
|COUNTRY_OF_ORIGIN|  END_DATE|HOSPITAL_ADDRESS|HOSPITAL_ID|       HOSPITAL_NAME|HOSPITAL_SHORT_NAME|      HOSPITAL_TYPE|      LAST_UPDATE_DT|LAST_UPDATE_USER|SOURCE_IDENTIFIER|          START_DATE|VERSION|                 _id|
+-----------------+----------+----------------+-----------+--------------------+-------------------+-------------------+--------------------+----------------+-----------------+--------------------+-------+--------------------+
|             null|      null|            null|          0|                null|               null|               null|                null|            null|             null|                null|    0.0|{61082d9721e1ff33...|
|               US|9999-12-31|          XYZ123|      H1001|BETHESDA MEMORIAL...|            

### Manufacturer Raw Staging

In [26]:
manufacturer.insert_one({'MANUFACTURER_ID':0, 'VERSION': 0})

In [27]:
# file_name = 'Manufacturer.csv'
# file_name = input('Enter File Name: ')
# manufacturer_csv = spark.read.csv(raw_data_path + file_name, header = True).dropna(how = "all")
# manufacturer_csv.show()

In [28]:
manufacturer_csv = spark.read.format("mongo").option("uri", 
                                "mongodb://127.0.0.1:27017/medexchain-catalog.manufacturer").load()
manufacturer_csv.createOrReplaceTempView('manufacturer_raw_view')

In [29]:
manufacturer_csv = manufacturer_csv.select(col('_id'),
                         col('org_type'),
                         col('name'),
                         col('short_name'),
                         col('source_identifier'),
                         col('country_of_origin'),
                         col('address'))

AnalysisException: cannot resolve '`_id`' given input columns: [];
'Project ['_id, 'org_type, 'name, 'short_name, 'source_identifier, 'country_of_origin, 'address]
+- Relation[] MongoRelation(MongoRDD[84] at RDD at MongoRDD.scala:51,Some(StructType()))


In [30]:
manufacturer_csv = manufacturer_csv.withColumnRenamed('_id', 'MANUFACTURER_ID').\
                                    withColumnRenamed('org_type', 'MANUFACTURER_ORG_TYPE').\
                                    withColumnRenamed('name', 'MANUFACTURER_NAME').\
                                    withColumnRenamed('short_name', 'MANUFACTURER_SHORT_NAME').\
                                    withColumnRenamed('source_identifier', 'SOURCE_IDENTIFIER').\
                                    withColumnRenamed('country_of_origin', 'COUNTRY_OF_ORIGIN').\
                                    withColumnRenamed('address', 'MANUFACTURER_ADDRESS')
#                                             withColumnRenamed('UM', 'UOM').\
#                                             withColumnRenamed('base_price', 'BASE_PRICE').\
#                                             withColumnRenamed(' PRODUCT_TYPE_ID ', 'PRODUCT_TYPE_ID').\
#                                             withColumnRenamed('name', 'NAME').\
#                                             withColumnRenamed('short_description', 'SHORT_DESCRIPTION').\
#                                             withColumnRenamed('status', 'PRODUCT_STATUS').\
#                                             withColumnRenamed('is_pack', 'IS_PACK').\
#                                             withColumnRenamed('units_in_pack', 'UNITS_IN_PACK').\
#                                             withColumnRenamed('category', 'PRODUCT_CATEGORY').\
#                                             withColumnRenamed('mfg_date', 'MFG_DATE').\
#                                             withColumnRenamed('expiry_date', 'EXPIRY_DATE').\
#                                             withColumnRenamed('image', 'PRODUCT_IMAGE')

In [31]:
manufacturer_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.MANUFACTURER").load()
manufacturer_data.createOrReplaceTempView('manufacturer_view')

In [32]:
for i in manufacturer_csv.toJSON().collect():
    new_record = json.loads(i)
    manufacturer_id = str(new_record['MANUFACTURER_ID'])

    query = "select * from manufacturer_view \
             where MANUFACTURER_ID = '" + manufacturer_id + "' and \
                   VERSION = (select max(VERSION) from manufacturer_view \
             where MANUFACTURER_ID = '" + manufacturer_id + "')"
    
    available_data = spark.sql(query)
    print(manufacturer_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        manufacturer.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['MANUFACTURER_NAME'] != available_data.rdd.collect()[0]['MANUFACTURER_NAME'])|\
         (new_record['MANUFACTURER_ADDRESS'] != available_data.rdd.collect()[0]['MANUFACTURER_ADDRESS'])|\
         (new_record['MANUFACTURER_SHORT_NAME'] != available_data.rdd.collect()[0]['MANUFACTURER_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.
         
         manufacturer.update_many({'MANUFACTURER_ID': manufacturer_id , 
                                   'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                  {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                            'END_DATE': str(datetime.now())}})
        
         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         manufacturer.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                 
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        manufacturer.update_one({'MANUFACTURER_ID': manufacturer_id, 
                                 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.

In [33]:
manufacturer_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.MANUFACTURER").load()
manufacturer_data.show()                                

+-----------------+----------+--------------------+----------------+--------------------+---------------+--------------------+---------------------+-----------------------+-----------------+--------------------+-------+--------------------+
|COUNTRY_OF_ORIGIN|  END_DATE|      LAST_UPDATE_DT|LAST_UPDATE_USER|MANUFACTURER_ADDRESS|MANUFACTURER_ID|   MANUFACTURER_NAME|MANUFACTURER_ORG_TYPE|MANUFACTURER_SHORT_NAME|SOURCE_IDENTIFIER|          START_DATE|VERSION|                 _id|
+-----------------+----------+--------------------+----------------+--------------------+---------------+--------------------+---------------------+-----------------------+-----------------+--------------------+-------+--------------------+
|             null|      null|                null|            null|                null|              0|                null|                 null|                   null|             null|                null|    0.0|{61082da221e1ff33...|
|               US|9999-12-31|2021-0

### User-Roles Raw Staging

In [34]:
user_roles.insert_one({'USER_ID': 0, 'VERSION': 0})

In [35]:
file_name = 'User.csv'
# file_name = input('Enter File Name: ')
user_csv = spark.read.csv(raw_data_path + file_name, header = True).dropna(how = "all")
# user_csv.show()

In [36]:
user_data = spark.read.format("mongo").option("uri",
                            "mongodb://127.0.0.1:27017/MED_EX_CHAIN.USER_ROLES").load()
user_data.createOrReplaceTempView('user_view')

In [37]:
for i in user_csv.toJSON().collect():
    new_record = json.loads(i)
    user_id = new_record['USER_ID']

    query = "select * from user_view \
             where USER_ID = '" + user_id + "' and\
                   VERSION = (select max(VERSION) from user_view \
             where USER_ID = '" + user_id + "')"
    
    available_data = spark.sql(query)
    print(user_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        user_roles.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['FIRST_NAME'] != available_data.rdd.collect()[0]['FIRST_NAME'])|\
         (new_record['USER_ADDRESS'] != available_data.rdd.collect()[0]['USER_ADDRESS'])|\
         (new_record['PHONE_NUMBER'] != available_data.rdd.collect()[0]['PHONE_NUMBER'])|\
	     (new_record['EMAIL_ID'] != available_data.rdd.collect()[0]['EMAIL_ID']): 
         # add conditions for all the columns which are eligible for the SCD.

         user_roles.update_many({'USER_ID': user_id, 
                                 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                  {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                            'END_DATE': str(datetime.now())}})
         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         user_roles.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                 
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        user_roles.update_one({'USER_ID': user_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


U1001 1
update record with same latest version
U1002 1
update record with same latest version


In [38]:
user_data = spark.read.format("mongo").option("uri",
                            "mongodb://127.0.0.1:27017/MED_EX_CHAIN.USER_ROLES").load()
# user_data.show()                            

## CATALOG DATA TO MONGODB STAGING

In [39]:
product_catalog_data_path = "../Data/SampleData/Product Data"

### Distributor Product Catalog

In [40]:
distributor_file = 'CardinalHealth _Product_Catalog.xlsx'
# distributor_file = input('Enter distributor product catalog file name: ')
distributor_catalog = pd.read_excel(product_catalog_data_path + "/Distributor/" + distributor_file).dropna(how = 'all').astype(str)

FileNotFoundError: [Errno 2] No such file or directory: '../Data/SampleData/Product Data/Distributor/CardinalHealth _Product_Catalog.xlsx'

In [ ]:
distributor_catalog = pd.read_excel(product_catalog_data_path + "/Distributor/Mckesson_Product_Catalog.xlsx", 
                                    sheet_name = 'Charting Paper Detail').dropna(how = 'all').astype(str)

In [ ]:
distr_product_catalog = distributor_catalog[['Account Name', 'Supplier Name', 'Supplier SKU Number', 
                                             'Distributor SKU', 'SKU Description', 'Hospital Item Number',                                                        'Distributor Name', 'Contract Price', 'Contract Number', 'Unit Price']]

distr_product_catalog = distr_product_catalog.rename(columns = {'Account Name': 'HOSPITAL_NAME',
                                                                'Supplier Name': 'MANUFACTURER_NAME',
                                                                'Supplier SKU Number': 'PRODUCT_ID',
                                                                'Distributor SKU': 'DISTRIBUTOR_SKU',
                                                                'SKU Description': 'DESCRIPTION',
                                                                'Hospital Item Number': 'HOSPITAL_SKU',
                                                                'Distributor Name': 'DISTRIBUTOR_NAME',
                                                                'Contract Price':'CONTRACT_PRICE',
                                                                'Contract Number':'CONTRACT_NUMBER',
                                                                'Unit Price':'BASE_PRICE'})

correlation_data = distr_product_catalog[['PRODUCT_ID', 'DISTRIBUTOR_NAME', 'DISTRIBUTOR_SKU', 'HOSPITAL_NAME',                                                'HOSPITAL_SKU', 'MANUFACTURER_NAME', 'DESCRIPTION', 'CONTRACT_PRICE',                                                'CONTRACT_NUMBER','BASE_PRICE']]


correlation.insert_many(json.loads(correlation_data.to_json(orient = 'records')))

### Manufacturer Product Catalog

In [ ]:
product.insert_one({'PRODUCT_ID': 0, 'VERSION': 0})

In [ ]:
# file_name = 'Medical Action Industries_product_catalog_1.csv'
# file_name = 'MALLINCKRODT_product_catalog.csv'
file_name = 'Manufacturer-ProductSimilarity_Catalogue.csv'

# file_name = input('Enter Manufacturer product catalog File Name: ')
manufacturer_catalog = spark.read.csv(product_catalog_data_path + "/Manufacturer/" + file_name, 
                                        header = True).dropna(how = 'all')
# manufacturer_catalog.show()

In [ ]:
manufacturer_catalog = spark.read.format("mongo").option("uri", 
                                "mongodb://127.0.0.1:27017/medexchain-catalog.product").load()
manufacturer_catalog.createOrReplaceTempView('product_raw_view')

In [ ]:
manufacturer_catalog.show()

In [ ]:
manufacturer_catalog = manufacturer_catalog.select(col('item_number'),
                                                   col('manufacturer_catalog_number'),
                                                   col('description'),
                                                   col('size'),
                                                   col('gauge'),
                                                   col('color'),
                                                   col('BUOM'),
                                                   col('UM'),
                                                   col('base_price'),
                                                   col('type_id'),
                                                   col('name'),
                                                   col('short_description'),
                                                   col('status'),
                                                   col('is_pack'),
                                                   col('units_in_pack'),
                                                   col('category'),
                                                   col('mfg_date'),
                                                   col('expiry_date'),
                                                   col('image'),
                                                   col('manufacturer'))

In [41]:
manufacturer_catalog = manufacturer_catalog.withColumnRenamed('item_number', 'PRODUCT_ID').\
                                            withColumnRenamed('description', 'DESCRIPTION').\
                                            withColumnRenamed('manufacturer', 'MANUFACTURER').\
                                            withColumnRenamed('size', 'SIZE').\
                                            withColumnRenamed('manufacturer_catalog_number', 'CATALOG_NUMBER').\
                                            withColumnRenamed('gauge', 'VOLUME').\
                                            withColumnRenamed('color', 'COLOR').\
                                            withColumnRenamed('BUOM', 'WEIGHT_UOM').\
                                            withColumnRenamed('UM', 'UOM').\
                                            withColumnRenamed('base_price', 'BASE_PRICE').\
                                            withColumnRenamed('type_id', 'PRODUCT_TYPE_ID').\
                                            withColumnRenamed('name', 'NAME').\
                                            withColumnRenamed('short_description', 'SHORT_DESCRIPTION').\
                                            withColumnRenamed('status', 'PRODUCT_STATUS').\
                                            withColumnRenamed('is_pack', 'IS_PACK').\
                                            withColumnRenamed('units_in_pack', 'UNITS_IN_PACK').\
                                            withColumnRenamed('category', 'PRODUCT_CATEGORY').\
                                            withColumnRenamed('mfg_date', 'MFG_DATE').\
                                            withColumnRenamed('expiry_date', 'EXPIRY_DATE').\
                                            withColumnRenamed('image', 'PRODUCT_IMAGE')

NameError: name 'manufacturer_catalog' is not defined

In [42]:
product_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT").load()
product_data.createOrReplaceTempView('product_view')

In [43]:
for i in manufacturer_catalog.toJSON().collect():
    new_record = json.loads(i)
    product_id = new_record['PRODUCT_ID']

    query = "select * from product_view \
             where PRODUCT_ID = '" + product_id + "' and \
                   VERSION = (select max(VERSION) from product_view \
             where PRODUCT_ID = '" + product_id + "')"
    
    available_data = spark.sql(query)
    print(product_id, available_data.count())
    
    # no data available, insert with version 1.0
    if available_data.count() == 0: 
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        product.insert_one(insert_data)
        print('insert data with 1 version.')
    
    # insert data with new version.    
    elif (new_record['NAME'] != available_data.rdd.collect()[0]['NAME'])|\
         (new_record['DESCRIPTION'] != available_data.rdd.collect()[0]['DESCRIPTION'])|\
         (new_record['BASE_PRICE'] != available_data.rdd.collect()[0]['BASE_PRICE'])|\
         (new_record['CATALOG_NUMBER'] != available_data.rdd.collect()[0]['CATALOG_NUMBER'])|\
         (new_record['UNITS_IN_PACK'] != available_data.rdd.collect()[0]['UNITS_IN_PACK'])|\
         (new_record['SHORT_DESCRIPTION'] != available_data.rdd.collect()[0]['SHORT_DESCRIPTION']): 
         # add conditions for all the columns which are eligible for the SCD.

         product.update_many({'PRODUCT_ID': product_id,
                              'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                             {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                       'END_DATE':str(datetime.now())}})

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         product.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
                
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        product.update_one({'PRODUCT_ID': product_id, 
                            'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                           {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


NameError: name 'manufacturer_catalog' is not defined

In [44]:
product_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT").load()
product_data.show()                                

+----------+--------------+-----+--------------------+----------+-----------+-------+--------------------+--------------------+--------+--------------------+----------------+----------+-------------+--------------+--------------------+------+--------------------+-------------+----+-------+------+----------+--------------------+
|BASE_PRICE|CATALOG_NUMBER|COLOR|         DESCRIPTION|  END_DATE|EXPIRY_DATE|IS_PACK|      LAST_UPDATE_DT|        MANUFACTURER|MFG_DATE|                NAME|PRODUCT_CATEGORY|PRODUCT_ID|PRODUCT_IMAGE|PRODUCT_STATUS|   SHORT_DESCRIPTION|  SIZE|          START_DATE|UNITS_IN_PACK| UOM|VERSION|VOLUME|WEIGHT_UOM|                 _id|
+----------+--------------+-----+--------------------+----------+-----------+-------+--------------------+--------------------+--------+--------------------+----------------+----------+-------------+--------------+--------------------+------+--------------------+-------------+----+-------+------+----------+--------------------+
|      nul